In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import tensorflow
from tensorflow.keras import datasets
import cv2
from tensorflow.keras import applications


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()

In [ ]:
label = {0: 'T-shirt/top',
1:'Trouser',
2:'Pullover',
3:'Dress',
4:'Coat',
5:'Sandal',
6:'Shirt',
7:'Sneaker',
8:'Bag',
9:'Ankle boot'}

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(5, 5, figsize=(10, 12))
for i in range(25):
    ax[i // 5, i % 5].imshow(train_images[i])
    ax[i // 5, i % 5].set_title(label[train_labels[i]])

In [ ]:
bat_size = 32
image_size = 28
image_channel = 1

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
train_labels_ohe = enc.fit_transform(train_labels.reshape(-1, 1)).toarray().astype(int)
test_labels_ohe = enc.transform(test_labels.reshape(-1, 1)).toarray().astype(int)

In [ ]:
from sklearn.model_selection import train_test_split 
x_te, x_val, y_te, y_val = train_test_split(test_images, test_labels_ohe, test_size=0.3, random_state=21)

In [ ]:
train_images = train_images.reshape((-1, 28, 28, 1))
x_val = x_val.reshape((-1, 28, 28, 1))
x_te  = x_te.reshape((-1, 28, 28, 1))

In [ ]:
y_val[1].shape

In [ ]:
from keras import Model, layers


net = {}

net['input'] = layers.Input((28, 28))
net['flat'] = layers.Flatten()(net['input'])
net['dense_1'] = layers.Dense(256, activation = 'relu')(net['flat'])
net['dense_2'] = layers.Dense(128, activation = 'selu')(net['dense_1'])
net['dense_3'] = layers.Dense(64, activation = 'relu')(net['dense_2'])
net['output'] = layers.Dense(10,activation = 'softmax')(net['dense_3'])


model = Model(net['input'], net['output'])


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
train_labels[0].shape

In [ ]:
n_epochs, n_save_after = 100, 80


for i in range(n_epochs):
    model.fit(train_images, train_labels_ohe, epochs=1, verbose=0, validation_data=(x_val, y_val))
    print(f'step : {i}' )
    if i >= n_save_after:
        print(f'step : {i}' )
        model.save('/kaggle/working/model_' + str(i) + '.h5')

In [ ]:
from tensorflow.keras.models import load_model 

def load_all_models(n_start, n_end):
    all_models = list()
    
    for epoch in range(n_start, n_end):
        filename = '/kaggle/working/model_' + str(epoch) + '.h5'
        model = load_model(filename)
        all_models.append(model)
        print('>loaded %s' % filename)
        
    return all_models



members = load_all_models(n_save_after, n_epochs)

print('Loaded %d models' % len(members))


members = list(reversed(members))

In [ ]:
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

def ensemble_predictions(members, test_images):
    
    
    yhats = [model.predict(test_images) for model in members]
    yhats = np.array(yhats)
    summed = np.sum(yhats, axis=0)
    result = np.argmax(summed, axis=1)
    
    return result

def evaluate_n_members(members, n_members, test_images, testy):
    
    
    subset = members[:n_members]
    yhat = ensemble_predictions(subset, test_images)
    return accuracy_score(testy, yhat)



In [ ]:
single_scores, ensemble_scores = list(), list()


for i in range(1, len(members)+1):
    ensemble_score = evaluate_n_members(members, i, test_images, test_labels)
    testy_enc = to_categorical(test_labels)
    
    
    _, single_score = members[i-1].evaluate(test_images, testy_enc, verbose=0)
    print('> %d: single=%.3f, ensemble=%.3f' % (i, single_score, ensemble_score))
    
    
    ensemble_scores.append(ensemble_score)
    single_scores.append(single_score)

In [ ]:
print('Single score accuracy %.3f (%.3f)' % (np.mean(single_scores), np.std(single_scores)))
print('Ensemble model accuracy %.3f (%.3f)' % (np.mean(ensemble_scores), np.std(ensemble_scores)))